In [23]:
import pandas as pd
import glob

#Using glob to look at all the csv files
csv_files = glob.glob("*.csv")
print(csv_files)
print("")
#Looking at one csv file
csv_file = pd.read_csv("used_car_prices1.csv")
print(csv_file)


['used_car_prices1.csv', 'used_car_prices2.csv', 'used_car_prices3.csv']

       car_model  year_of_manufacture         price    fuel
0           ritz                 2014   5000.000000  Petrol
1            sx4                 2013   7089.552239  Diesel
2           ciaz                 2017  10820.895522  Petrol
3        wagon r                 2011   4253.731343  Petrol
4          swift                 2014   6865.671642  Diesel
5  vitara brezza                 2018  13805.970149  Diesel
6           ciaz                 2015  10074.626866  Petrol
7        s cross                 2015   9701.492537  Diesel
8           ciaz                 2016  13059.701493  Diesel
9           ciaz                 2015  11119.402985  Diesel


In [25]:
#Using glob to look at all the json files
csv_files = glob.glob("*.json")
print(csv_files)
print("")
#Looking at one json file
csv_file = pd.read_csv("used_car_prices1.json")
print(csv_file)

['used_car_prices1.json', 'used_car_prices2.json', 'used_car_prices3.json']

       {"car_model":"ritz"  year_of_manufacture:2012  price:4626.8656716418  \
0      {"car_model":"ritz"  year_of_manufacture:2011  price:3507.4626865672   
1     {"car_model":"swift"  year_of_manufacture:2014  price:7388.0597014925   
2    {"car_model":"ertiga"  year_of_manufacture:2014   price:8955.223880597   
3     {"car_model":"dzire"  year_of_manufacture:2014  price:8208.9552238806   
4       {"car_model":"sx4"  year_of_manufacture:2011  price:4402.9850746269   
5     {"car_model":"dzire"  year_of_manufacture:2015  price:6940.2985074627   
6       {"car_model":"800"  year_of_manufacture:2003   price:522.3880597015   
7  {"car_model":"alto k10"  year_of_manufacture:2016  price:4477.6119402985   
8       {"car_model":"sx4"  year_of_manufacture:2003  price:3358.2089552239   

   fuel:"Diesel"}  
0  fuel:"Petrol"}  
1  fuel:"Diesel"}  
2  fuel:"Diesel"}  
3  fuel:"Diesel"}  
4     fuel:"CNG"}  
5  fuel:"Pet

In [26]:
#Using glob to look at all the xml files
csv_files = glob.glob("*.xml")
print(csv_files)
print("")
#Looking at one json file
csv_file = pd.read_csv("used_car_prices1.xml")
print(csv_file)

['used_car_prices1.xml', 'used_car_prices2.xml', 'used_car_prices3.xml']

              <?xml version="1.0" encoding="UTF-8" ?>
0                                              <root>
1                                               <row>
2                <car_model>corolla altis</car_model>
3       <year_of_manufacture>2013</year_of_manufac...
4                   <price>10373.134328358208</price>
..                                                ...
57      <year_of_manufacture>2016</year_of_manufac...
58                   <price>21268.65671641791</price>
59                                <fuel>Petrol</fuel>
60                                             </row>
61                                            </root>

[62 rows x 1 columns]


In [35]:
#Function to read csv files
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

#Function to read json files
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process, lines = True)
    return dataframe

#Use xml.etree.ElementTree to read xml 
import xml.etree.ElementTree as ET

def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=["car_model","year_of_manufacture","price","fuel"])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
#Loop through each car    
    for car in root:
        car_model = car.find("car_model").text
        year_of_manufacture = car.find("year_of_manufacture").text
        price = float(car.find("price").text)
        fuel = car.find("fuel").text
        dataframe = pd.concat([dataframe, pd.DataFrame([{"car_model":car_model,"year_of_manufacture":year_of_manufacture, "price":price,"fuel": fuel}])], ignore_index=True) 
        return dataframe    

In [31]:
log_file = "log_file.txt" #Logging process
target_file = "transformed_data.csv" #Finale data 

#Function to extract
def extract(): 
    extracted_data = pd.DataFrame(columns=["car_model","year_of_manufacture","price","fuel"])  # create an empty data frame to hold extracted data
     
    # process all csv files, except the target file
    for csvfile in glob.glob("*.csv"): 
        if csvfile != target_file:  # check if the file is not the target file
            extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_csv(csvfile))], ignore_index=True) 
         
    # process all json files 
    for jsonfile in glob.glob("*.json"): 
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_json(jsonfile))], ignore_index=True) 
     
    # process all xml files 
    for xmlfile in glob.glob("*.xml"): 
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_xml(xmlfile))], ignore_index=True) 
         
    return extracted_data     


In [37]:
from datetime import datetime 

def transform(data):
    data['price'] = round(data.price, 2)
    return data

def load_data(target_file, transformed_data): 
    transformed_data.to_csv(target_file) 
  
def log_progress(message): 
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open(log_file,"a") as f: 
        f.write(timestamp + ',' + message + '\n') 
  
# Log the initialization of the ETL process 
log_progress("ETL Job Started") 
  
# Log the beginning of the Extraction process 
log_progress("Extract phase Started") 
extracted_data = extract() 
  
# Log the completion of the Extraction process 
log_progress("Extract phase Ended") 
  
# Log the beginning of the Transformation process 
log_progress("Transform phase Started") 
transformed_data = transform(extracted_data) 
print("Transformed Data") 
print(transformed_data) 
  
# Log the completion of the Transformation process 
log_progress("Transform phase Ended") 
  
# Log the beginning of the Loading process 
log_progress("Load phase Started") 
load_data(target_file,transformed_data) 
  
# Log the completion of the Loading process 
log_progress("Load phase Ended") 
  
# Log the completion of the ETL process 
log_progress("ETL Job Ended") 

Transformed Data
        car_model year_of_manufacture     price    fuel
0            ritz                2014   5000.00  Petrol
1             sx4                2013   7089.55  Diesel
2            ciaz                2017  10820.90  Petrol
3         wagon r                2011   4253.73  Petrol
4           swift                2014   6865.67  Diesel
..            ...                 ...       ...     ...
58        etios g                2014   6119.40  Petrol
59       fortuner                2014  29835.82  Diesel
60  corolla altis                2013  10373.13  Petrol
61     etios liva                2014   5895.52  Diesel
62  corolla altis                2016  21985.07  Diesel

[63 rows x 4 columns]


C:\Users\16828\AppData\Local\Temp\ipykernel_42576\365286566.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_csv(csvfile))], ignore_index=True)
C:\Users\16828\AppData\Local\Temp\ipykernel_42576\159650625.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe = pd.concat([dataframe, pd.DataFrame([{"car_model":car_model,"year_of_manufacture":year_of_manufacture, "price":price,"fuel": fuel}])], ignore_index=True)
C:\U

In [45]:
df = pd.read_csv(target_file)
print(df)

df2 = pd.read_csv("log_file.txt")
print(df2)

    Unnamed: 0      car_model  year_of_manufacture     price    fuel
0            0           ritz                 2014   5000.00  Petrol
1            1            sx4                 2013   7089.55  Diesel
2            2           ciaz                 2017  10820.90  Petrol
3            3        wagon r                 2011   4253.73  Petrol
4            4          swift                 2014   6865.67  Diesel
..         ...            ...                  ...       ...     ...
58          58        etios g                 2014   6119.40  Petrol
59          59       fortuner                 2014  29835.82  Diesel
60          60  corolla altis                 2013  10373.13  Petrol
61          61     etios liva                 2014   5895.52  Diesel
62          62  corolla altis                 2016  21985.07  Diesel

[63 rows x 5 columns]
   2025-Aug-31-21:33:56          ETL Job Started
0  2025-Aug-31-21:33:56    Extract phase Started
1  2025-Aug-31-21:33:56      Extract phase Ended
2 